In [38]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('image', cmap='gray')
mpl.rcParams["image.interpolation"] = 'none'
%matplotlib inline
import matplotlib.animation

from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import random

%config InlineBackend.figure_format = 'retina'
from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from stardist.matching import matching, matching_dataset
from stardist.models import Config2D, StarDist2D, StarDistData2D
from csbdeep.utils import Path, normalize
import skimage
from tifffile import imsave, imread
from glob import glob

np.random.seed(42)
lbl_cmap = random_label_cmap()

# BUILD TRAIN DATASET

In [ ]:
def get_frame(cap, frame, x1, y1, x2, y2, w, h, preprocess):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
    ret, image = cap.read()
    if preprocess:
        npImage = np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
        alpha = Image.new('L', (w, h), 0)
        draw = ImageDraw.Draw(alpha)
        draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)
        npAlpha = np.array(alpha)
        npImage = npImage*npAlpha
        ind = np.where(npImage == 0)
        npImage[ind] = npImage[200, 200]
        npImage = npImage[y1:y2, x1:x2]
        return npImage
    elif not preprocess:
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        raise ValueError("preprocess must be a boolean")

In [39]:
trajectories = pd.read_parquet('./49b_1r/old/tracking_data/hough/tracking_hough_trackpy_linking.parquet')
# rename column to be consistent with the other datasets
trajectories.rename(columns={'d':'r'}, inplace=True)
# remove rows with NaN values in the radius column
trajectories.dropna(subset=['r'], inplace=True)
display(trajectories.head())

,x,y,r,frame,particle,flag,color
0,417.0,174.0,20.315157,0.0,0,0,#8B0E71
1,505.5,529.5,20.472479,0.0,1,0,#53BF7C
2,555.0,528.0,20.537350,0.0,2,0,#3706D8
3,759.0,538.5,20.583065,0.0,3,0,#5C524E
4,172.5,559.5,20.962828,0.0,4,0,#440066


In [42]:
video = cv2.VideoCapture('/Users/matteoscandola/MasterThesis/tracking/data/49b1r.mp4')
video.set(cv2.CAP_PROP_POS_FRAMES, 0)
_, first_frame = video.read()

In [43]:
# sample x frames from the hough-method tracking dataset
n_samples = 1000
np.random.seed(42)
sample_frames = np.sort(np.random.choice(trajectories.frame.unique().astype(int), size=n_samples, replace=False))

for frame in tqdm(sample_frames):
    frame_img = get_frame(video, frame, 55, 55, 880, 880, 920, 960, True)
    imsave(f"/Users/matteoscandola/MasterThesis/tracking/train_49b-1r/image/49b1r_frame{frame}.tif", frame_img)
    df = trajectories.loc[(trajectories.frame == frame), ["x", "y", "r"]]
    image = np.zeros((880 - 55, 880 - 55), dtype=np.uint8)
    count = 0
    for _, droplet in df.iterrows():
        x, y, radius = droplet['x'], droplet['y'], droplet['r']
        cv2.circle(image, (int(x)- 55, int(y) - 55), int(radius), int(count), thickness=-1)
        count += 1
    imsave(f"/Users/matteoscandola/MasterThesis/tracking/train_49b-1r/mask/49b1r_frame{frame}.tif", image)    

100%|██████████| 1000/1000 [09:51<00:00,  1.69it/s]


# TRAIN

In [ ]:
X = sorted(glob("./train_49b-1r/image/*.tif"))
Y = sorted(glob("./train_49b-1r/mask/*.tif"))
assert all(Path(x).name==Path(y).name for x,y in zip(X,Y))

X = list(map(imread,X))
Y = list(map(imread,Y))
n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]

axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    sys.stdout.flush()

X = [normalize(x,1,99.8,axis=axis_norm) for x in tqdm(X)]
Y = [fill_label_holes(y) for y in tqdm(Y)]

KeyboardInterrupt: 

In [ ]:
assert len(X) > 1, "not enough training data"
rng = np.random.RandomState(42)
ind = rng.permutation(len(X))
n_val = max(1, int(round(0.15 * len(ind))))
ind_train, ind_val = ind[:-n_val], ind[-n_val:]
X_val, Y_val = [X[i] for i in ind_val]  , [Y[i] for i in ind_val]
X_trn, Y_trn = [X[i] for i in ind_train], [Y[i] for i in ind_train] 
print('number of images: %3d' % len(X))
print('- training:       %3d' % len(X_trn))
print('- validation:     %3d' % len(X_val))

In [ ]:
def plot_img_label(img, lbl, img_title="image", lbl_title="label", **kwargs):
    fig, (ai,al) = plt.subplots(1,2, figsize=(12,5), gridspec_kw=dict(width_ratios=(1.25,1)))
    im = ai.imshow(img, cmap='gray', clim=(0,1))
    ai.set_title(img_title)    
    fig.colorbar(im, ax=ai)
    al.imshow(lbl, cmap=lbl_cmap)
    al.set_title(lbl_title)
    plt.tight_layout()

In [ ]:
i = min(9, len(X)-1)
img, lbl = X[i], Y[i]
assert img.ndim in (2,3)
img = img if (img.ndim==2 or img.shape[-1]==3) else img[...,0]
plot_img_label(img,lbl)
None;

In [ ]:
gputools_available()

In [ ]:
# 32 is a good default choice (see 1_data.ipynb)
n_rays = 32

# Use OpenCL-based computations for data generator during training (requires 'gputools')
use_gpu = False and gputools_available()

# Predict on subsampled grid for increased efficiency and larger field of view
grid = (2,2)

conf = Config2D (
    n_rays       = n_rays,
    grid         = grid,
    use_gpu      = use_gpu,
    n_channel_in = n_channel,
)
print(conf)
vars(conf)

In [ ]:
if use_gpu:
    from csbdeep.utils.tf import limit_gpu_memory
    # adjust as necessary: limit GPU memory to be used by TensorFlow to leave some to OpenCL-based computations
    limit_gpu_memory(0.8)
    # alternatively, try this:
    # limit_gpu_memory(None, allow_growth=True)

In [ ]:
model = StarDist2D(conf, name='stardist', basedir='models')

In [ ]:
median_size = calculate_extents(list(Y), np.median)
fov = np.array(model._axes_tile_overlap('YX'))
print(f"median object size:      {median_size}")
print(f"network field of view :  {fov}")
if any(median_size > fov):
    print("WARNING: median object size larger than field of view of the neural network.")

In [ ]:
def random_fliprot(img, mask): 
    assert img.ndim >= mask.ndim
    axes = tuple(range(mask.ndim))
    perm = tuple(np.random.permutation(axes))
    img = img.transpose(perm + tuple(range(mask.ndim, img.ndim))) 
    mask = mask.transpose(perm) 
    for ax in axes: 
        if np.random.rand() > 0.5:
            img = np.flip(img, axis=ax)
            mask = np.flip(mask, axis=ax)
    return img, mask 

def random_intensity_change(img):
    img = img*np.random.uniform(0.6,2) + np.random.uniform(-0.2,0.2)
    return img


def augmenter(x, y):
    """Augmentation of a single input/label image pair.
    x is an input image
    y is the corresponding ground-truth label image
    """
    x, y = random_fliprot(x, y)
    x = random_intensity_change(x)
    # add some gaussian noise
    sig = 0.02*np.random.uniform(0,1)
    x = x + sig*np.random.normal(0,1,x.shape)
    return x, y

In [ ]:
# plot some augmented examples
img, lbl = X[0],Y[0]
plot_img_label(img, lbl)
for _ in range(3):
    img_aug, lbl_aug = augmenter(img,lbl)
    plot_img_label(img_aug, lbl_aug, img_title="image augmented", lbl_title="label augmented")

In [ ]:
%load_ext tensorboard

In [ ]:
quick_demo = False

if quick_demo:
    print (
        "NOTE: This is only for a quick demonstration!\n"
        "      Please set the variable 'quick_demo = False' for proper (long) training.",
        file=sys.stderr, flush=True
    )
    model.train(X_trn, Y_trn, validation_data=(X_val,Y_val), augmenter=augmenter,
                epochs=2, steps_per_epoch=10)

    print("====> Stopping training and loading previously trained demo model from disk.", file=sys.stderr, flush=True)
    model = StarDist2D.from_pretrained('2D_demo')
else:
    model.train(X_trn, Y_trn, validation_data=(X_val,Y_val), augmenter=augmenter)
None;